<a href="https://colab.research.google.com/github/soumilhooda/MLDLNNtoCV/blob/main/Q3_NNFL_Assignment2_SoumilHooda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
def Randomize(data):
  np.random.shuffle(data)
  return data

In [3]:
def data_split(data,train_ratio,val_ratio):
  X=data[:data.shape[0]-1,:]
  Y=data[data.shape[0]-1:,:]
  X_tr=X[:,:int(train_ratio*X.shape[1])]
  Y_tr=Y[:,:int(train_ratio*Y.shape[1])]
  X_val=X[:,int(train_ratio*X.shape[1]):int((train_ratio+val_ratio)*X.shape[1])]
  Y_val=Y[:,int(train_ratio*Y.shape[1]):int((train_ratio+val_ratio)*Y.shape[1])]
  X_te=X[:,int((train_ratio+val_ratio)*X.shape[1]):]
  Y_te=Y[:,int((train_ratio+val_ratio)*Y.shape[1]):]
  return X_tr,X_val,X_te,Y_tr,Y_val,Y_te

In [4]:
def Normalize(X_tr,X_val,X_te):
  mean=X_tr.mean(axis=1)
  std=X_tr.std(axis=1)
  for i in range(X_tr.shape[0]):
    X_tr[i:i+1,:]=(X_tr[i:i+1,:]-mean[i])/std[i]
    X_val[i:i+1,:]=(X_val[i:i+1,:]-mean[i])/std[i]
    X_te[i:i+1,:]=(X_te[i:i+1,:]-mean[i])/std[i]
  return X_tr,X_val,X_te

In [5]:
def hot_encoding(Y,n_Y):
  Y_new=np.zeros((n_Y,Y.shape[1]))
  for i in range(Y.shape[1]):
    Y_new[int(Y[0,i])-1][i]=1
  return Y_new

In [6]:
def preprocess(data):
  data=df.values
  data=Randomize(data)
  data=data.T
  X_tr,X_val,X_te,Y_tr,Y_val,Y_te=data_split(data,0.7,0.1)
  X_tr,X_val,X_te=Normalize(X_tr,X_val,X_te)
  Y_tr=hot_encoding(Y_tr,3)
  Y_val=hot_encoding(Y_val,3)
  Y_te=hot_encoding(Y_te,3)
  return X_tr,X_val,X_te,Y_tr,Y_val,Y_te

In [7]:
def sigmoid(Z):
  A=1/(1+np.exp(-Z))
  return A

In [8]:
def tanh(Z):
  A=(np.exp(Z)-np.exp(-Z))/(np.exp(Z)+np.exp(-Z))
  return A

In [9]:
def relu(Z):
  return(np.maximum(0,Z))

In [10]:
def init_para(n_X,n_h1,n_h2,n_Y):
  W1=np.random.randn(n_h1,n_X)
  b1=np.zeros((n_h1,1))
  W2=np.random.randn(n_h2,n_h1)
  b2=np.zeros((n_h2,1))
  W3=np.random.randn(n_Y,n_h2)
  b3=np.zeros((n_Y,1))
  para={"W1":W1,
              "b1":b1,
              "W2":W2,
              "b2":b2,
              "W3":W3,
              "b3":b3}
  return para

In [11]:
def forward_propagation(X,para,activation):
  Z1=np.dot(para["W1"],X)+para["b1"]
  if activation=="sigmoid":
    A1=sigmoid(Z1)
  elif activation=="tanh":
    A1=tanh(Z1)
  elif activation=="relu":
    A1=relu(Z1)
  Z2=np.dot(para["W2"],A1)+para["b2"]
  if activation=="sigmoid":
    A2=sigmoid(Z2)
  elif activation=="tanh":
    A2=tanh(Z2)
  elif activation=="relu":
    A2=relu(Z2)
  Z3=np.dot(para["W3"],A2)+para["b3"]
  Y_H=sigmoid(Z3)
  nodes={"Z1":Z1,
         "A1":A1,
         "Z2":Z2,
         "A2":A2,
         "Z3":Z3,
         "Y_H":Y_H}
  return nodes

In [12]:
def Cost(Y_H,Y):
  epsilon=1e-10
  m=Y.shape[1]
  cost=(-1/m)*np.sum(Y*np.log(Y_H+epsilon)+(1-Y)*np.log(1-Y_H+epsilon)) 
  return cost

In [13]:
def back_propagation(X,Y,para,nodes,activation,lam):
  m=X.shape[1]
  dZ3=nodes["Y_H"]-Y
  dW3=(1/m)*np.dot(dZ3,nodes["A2"].T)+(lam/m)*para["W3"]
  db3=(1/m)*np.sum(dZ3,axis=1,keepdims=True)
  dA2=np.dot(para["W3"].T,dZ3)
  if activation=="sigmoid":
    dZ2=dA2*nodes["A2"]*(1-nodes["A2"])
  elif activation=="tanh":
    dZ2=dA2*(1-nodes["A2"]**2)
  elif activation=="relu":
    dZ2=dA2*np.heaviside(nodes["Z2"],0)
  dW2=(1/m)*np.dot(dZ2,nodes["A1"].T)+(lam/m)*para["W2"]
  db2=(1/m)*np.sum(dZ2,axis=1,keepdims=True)
  dA1=np.dot(para["W2"].T,dZ2)
  if activation=="sigmoid":
    dZ1=dA1*nodes["A1"]*(1-nodes["A1"])
  elif activation=="tanh":
    dZ1=dA1*(1-nodes["A1"]**2)
  elif activation=="relu":
    dZ1=dA1*np.heaviside(nodes["Z1"],0)
  dW1=(1/m)*np.dot(dZ1,X.T)+(lam/m)*para["W1"]
  db1=(1/m)*np.sum(dZ1,axis=1,keepdims=True)
  grads={"dW1":dW1,
         "db1":db1,
         "dW2":dW2,
         "db2":db2,
         "dW3":dW3,
         "db3":db3}
  return grads

In [14]:
def gradient_descent(para,grads,learning_rate):
  para["W1"]=para["W1"]-learning_rate*grads["dW1"]
  para["b1"]=para["b1"]-learning_rate*grads["db1"]
  para["W2"]=para["W2"]-learning_rate*grads["dW2"]
  para["b2"]=para["b2"]-learning_rate*grads["db2"]
  para["W3"]=para["W3"]-learning_rate*grads["dW3"]
  para["b3"]=para["b3"]-learning_rate*grads["db3"]
  return para

In [15]:
def accuracy(Y_H,Y):
  a=np.abs(Y_H-Y)
  num_mistakes=(np.sum(a))/2
  acc=100*(Y.shape[1]-num_mistakes)/Y.shape[1]
  return acc

In [16]:
def return_max(Y_H):
  for i in range(Y_H.shape[1]):
    if Y_H[0][i]>Y_H[1][i] and Y_H[0][i]>Y_H[2][i]:
      Y_H[0][i]=1
      Y_H[1][i]=0
      Y_H[2][i]=0
    elif Y_H[1][i]>Y_H[0][i] and Y_H[1][i]>Y_H[2][i]:
      Y_H[0][i]=0
      Y_H[1][i]=1
      Y_H[2][i]=0
    else:
      Y_H[0][i]=0
      Y_H[1][i]=0
      Y_H[2][i]=1
  return Y_H

In [17]:
def model_train(X,Y,n_h1,n_h2,activation,lam):
  para=init_para(X.shape[0],n_h1,n_h2,Y.shape[0])
  for i in range(1000):
    nodes=forward_propagation(X,para,activation)
    grads=back_propagation(X,Y,para,nodes,activation,lam)
    para=gradient_descent(para,grads,0.1)
    if i%50==0:
      #print("Cost after iteration", i, ":", Cost(nodes["Y_H"],Y))
      Y_H=return_max(nodes["Y_H"])
      if accuracy(Y_H,Y)==100:
        #print("training accuracy :", accuracy(Y_H,Y), "%")
        return para,nodes
  Y_H=return_max(nodes["Y_H"])
  '''if activation=="tanh":
    print("training accuracy tanh :", accuracy(Y_H,Y), "%")
  elif activation=="relu":
    print("training accuracy relu :", accuracy(Y_H,Y), "%")
  elif activation=="sigmoid":
    print("training accuracy sigmoid :", accuracy(Y_H,Y), "%")'''  
  return para,nodes

In [18]:
def model_predict(X,Y,para,activation):
  nodes=forward_propagation(X,para,activation)
  Y_H=nodes["Y_H"]
  Y_H=return_max(Y_H)
  '''if activation=="tanh":
    print("validation accuracy tanh:", accuracy(Y_H,Y), "%")
  elif activation=="relu":
    print("validation accuracy relu:", accuracy(Y_H,Y), "%") 
  elif activation=="sigmoid":
    print("validation accuracy sigmoid :", accuracy(Y_H,Y), "%")'''  
  return accuracy(Y_H,Y)

In [19]:
from google.colab import files
uploaded = files.upload()

Saving data5.xlsx to data5.xlsx


In [20]:
df=pd.read_excel('data5.xlsx')
df

,15.260,14.840,0.871,5.763,3.312,2.221,5.220,1.000
0,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1.0
1,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1.0
2,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1.0
3,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1.0
4,14.38,14.21,0.8951,5.386,3.312,2.462,4.956,1.0
...,...,...,...,...,...,...,...,...
204,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3.0
205,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3.0
206,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3.0
207,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3.0


In [21]:
data=df.values
for h in range(3,6):
  sum_t=0
  sum_r=0
  sum_s=0
  tsum_t=0
  tsum_r=0
  tsum_s=0
  for n in range(5,10):
    np.random.seed(n)
    X_tr,X_val,X_te,Y_tr,Y_val,Y_te=preprocess(data)
    para_t,nodes_t=model_train(X_tr,Y_tr,5,h,"tanh",0)
    para_r,nodes_r=model_train(X_tr,Y_tr,5,h,"relu",0)
    para_s,nodes_s=model_train(X_tr,Y_tr,5,h,"sigmoid",0)
    tacc_t=model_predict(X_te,Y_te,para_t,"tanh")
    tacc_r=model_predict(X_te,Y_te,para_r,"relu")
    tacc_s=model_predict(X_te,Y_te,para_s,"sigmoid")
    acc_t=model_predict(X_val,Y_val,para_t,"tanh")
    acc_r=model_predict(X_val,Y_val,para_r,"relu")
    acc_s=model_predict(X_val,Y_val,para_s,"sigmoid")
    sum_t=sum_t+acc_t
    sum_r=sum_r+acc_r
    sum_s=sum_s+acc_s
    tsum_t=tsum_t+tacc_t
    tsum_r=tsum_r+tacc_r
    tsum_s=tsum_s+tacc_s
  
  print("final validation accuracy(tanh) of",h ,":",sum_t/5,"%")
  print("final validation accuracy(relu) of",h ,":", sum_r/5,"%")
  print("final validation accuracy of(sigmoid)",h ,":", sum_s/5,"%")
  print("final testing accuracy(tanh) of",h ,":",tsum_t/5,"%")
  print("final testing accuracy(relu) of",h ,":", tsum_r/5,"%")
  print("final testing accuracy of(sigmoid)",h ,":", tsum_s/5,"%")

final validation accuracy(tanh) of 3 : 98.0952380952381 %
final validation accuracy(relu) of 3 : 80.0 %
final validation accuracy of(sigmoid) 3 : 92.38095238095238 %
final testing accuracy(tanh) of 3 : 93.33333333333334 %
final testing accuracy(relu) of 3 : 80.0 %
final testing accuracy of(sigmoid) 3 : 91.42857142857143 %
final validation accuracy(tanh) of 4 : 92.38095238095238 %
final validation accuracy(relu) of 4 : 92.38095238095238 %
final validation accuracy of(sigmoid) 4 : 93.33333333333334 %
final testing accuracy(tanh) of 4 : 92.85714285714285 %
final testing accuracy(relu) of 4 : 92.38095238095238 %
final testing accuracy of(sigmoid) 4 : 91.42857142857144 %
final validation accuracy(tanh) of 5 : 88.57142857142858 %
final validation accuracy(relu) of 5 : 93.33333333333334 %
final validation accuracy of(sigmoid) 5 : 86.66666666666667 %
final testing accuracy(tanh) of 5 : 95.23809523809524 %
final testing accuracy(relu) of 5 : 93.80952380952381 %
final testing accuracy of(sigmoid

In [22]:
sum_val=0
sum_te=0
for n in range(5):
  np.random.seed(n)
  X_tr,X_val,X_te,Y_tr,Y_val,Y_te=preprocess(data)
  para_r,nodes_r=model_train(X_tr,Y_tr,5,4,"relu",0) 
  acc_val=model_predict(X_val,Y_val,para_r,"relu")
  acc_te=model_predict(X_te,Y_te,para_r,"relu")
  sum_val=sum_val+acc_val
  sum_te=sum_te+acc_te
print("final validation accuracy of:",sum_val/5,"%")  
print("final testing accuracy of:",sum_te/5,"%")

final validation accuracy of: 96.19047619047619 %
final testing accuracy of: 94.76190476190477 %
